In [1]:
#Installing libraries
!pip install vecstack

  Stored in directory: C:\Users\ENVY 15\AppData\Local\pip\Cache\wheels\5f\bb\4e\f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [13]:
import pandas as pd
import numpy as np
import requests
from io import StringIO 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, StackingRegressor,HistGradientBoostingRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from vecstack import stacking
from vecstack import StackingTransformer
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

In [3]:
#importing libraries
#
import pandas as pd
import numpy as np
import requests
from io import StringIO

In [4]:
#Creating submission and training dataframes
tra= pd.read_csv("Train.csv")
sub = pd.read_csv("SampleSubmission.csv")

In [5]:
#Separating the dataframe into testing and training datasets
precip_features_2019 = []
precip_features_2015 = []
for col in tra.columns:
    if '2019' in col:
        precip_features_2019.append(col)
    elif 'precip 2014' in col:
        precip_features_2015.append(col)
    elif 'precip 2015' in col:
        precip_features_2015.append(col)

In [6]:
#Separating the train dataset from the main dataframe
train = tra[tra.columns.difference(precip_features_2019)]
train.head()

,LC_Type1_mode,Square_ID,X,Y,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,...,precip 2015-01-11 - 2015-01-18,precip 2015-01-18 - 2015-01-25,precip 2015-01-25 - 2015-02-01,precip 2015-02-01 - 2015-02-08,precip 2015-02-08 - 2015-02-15,precip 2015-02-15 - 2015-02-22,precip 2015-02-22 - 2015-03-01,precip 2015-03-01 - 2015-03-08,precip 2015-03-08 - 2015-03-15,target_2015
0,9,4e3c3896-14ce-11ea-bce5-f49634744a41,34.26,-15.91,887.764222,0.0,0.0,0.0,14.844025,14.552823,...,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,0.0
1,9,4e3c3897-14ce-11ea-bce5-f49634744a41,34.26,-15.90,743.403912,0.0,0.0,0.0,14.844025,14.552823,...,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,0.0
2,9,4e3c3898-14ce-11ea-bce5-f49634744a41,34.26,-15.89,565.728343,0.0,0.0,0.0,14.844025,14.552823,...,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,0.0
3,10,4e3c3899-14ce-11ea-bce5-f49634744a41,34.26,-15.88,443.392774,0.0,0.0,0.0,14.844025,14.552823,...,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,0.0
4,10,4e3c389a-14ce-11ea-bce5-f49634744a41,34.26,-15.87,437.443429,0.0,0.0,0.0,14.844025,14.552823,...,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,0.0


In [7]:
#Separating the test dataset from the main dataframe
precip_features_2019.extend(['X','Y','elevation','LC_Type1_mode','Square_ID'])
test = tra[precip_features_2019] 

In [8]:
new_2015_cols = {}
for col,number in zip(precip_features_2015,range(1,len(precip_features_2015) + 1)):
    if 'precip' in col:
        new_2015_cols[col] = 'week_' +str(number) + '_precip'
new_2019_cols = {}
for col,number in zip(precip_features_2019,range(1,len(precip_features_2019) + 1)):
    if 'precip' in col:
        new_2019_cols[col] = 'week_' +str(number) + '_precip'
train.rename(columns = new_2015_cols,inplace =True)
test.rename(columns = new_2019_cols,inplace = True)

In [9]:
#Separating the target variable
target = train.target_2015

In [10]:
train,test = train.align(test,join = 'inner',axis = 1)
test.head()

,LC_Type1_mode,Square_ID,X,Y,elevation,week_1_precip,week_2_precip,week_3_precip,week_4_precip,week_5_precip,...,week_8_precip,week_9_precip,week_10_precip,week_11_precip,week_12_precip,week_13_precip,week_14_precip,week_15_precip,week_16_precip,week_17_precip
0,9,4e3c3896-14ce-11ea-bce5-f49634744a41,34.26,-15.91,887.764222,12.99262,4.582856,35.037532,4.796012,28.083314,...,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0
1,9,4e3c3897-14ce-11ea-bce5-f49634744a41,34.26,-15.90,743.403912,12.99262,4.582856,35.037532,4.796012,28.083314,...,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0
2,9,4e3c3898-14ce-11ea-bce5-f49634744a41,34.26,-15.89,565.728343,12.99262,4.582856,35.037532,4.796012,28.083314,...,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0
3,10,4e3c3899-14ce-11ea-bce5-f49634744a41,34.26,-15.88,443.392774,12.99262,4.582856,35.037532,4.796012,28.083314,...,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0
4,10,4e3c389a-14ce-11ea-bce5-f49634744a41,34.26,-15.87,437.443429,12.99262,4.582856,35.037532,4.796012,28.083314,...,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Splitting the data into training and testing dataframes
#
X = train.drop(['Square_ID'], axis = 1)  # Predictors
y = target                                                  # Target

sub = test.drop(['Square_ID'], axis = 1)           # Testing data

# Splitting the training dataset to 70%, and setting the random state to 90
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 90)

# Making predictions
#
predictions_cat = CatBoostRegressor(logging_level='Silent').fit(X_train, y_train).predict(sub)

In [15]:
# Using two different stacked ensembles to make predictions using the sklearn stacking regressor
#
X = train.drop(['Square_ID'], axis = 1)
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 90)

sub = test.drop(['Square_ID'], axis = 1)

estimators_1 = [
    ('xgb', XGBRegressor(objective ='reg:squarederror')),
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor()),
    ('lgb', LGBMRegressor()),
    ('svr', SVR()),
    ('lasso', Lasso()),
    ('kneiba', KNeighborsRegressor()),
    ('cat', CatBoostRegressor(logging_level='Silent'))
]

predictions_sreg = StackingRegressor(estimators=estimators_1, final_estimator=CatBoostRegressor(logging_level='Silent')).fit(X_train, y_train).predict(sub)


estimators_2 = [
    ('XBRF', XGBRFRegressor(objective ='reg:squarederror')),
    ('Bayesian', BayesianRidge()),
    ('ExtraTrees', ExtraTreesRegressor()),
    ('HistGradient', HistGradientBoostingRegressor()),
    ('NuSVR', NuSVR()),
    ('Ridge', Ridge()),
    ('KNeiba', KNeighborsRegressor()),
    ('cat', CatBoostRegressor(logging_level='Silent'))
]

predictions_sreg_2 = StackingRegressor(estimators=estimators_2, final_estimator=CatBoostRegressor(logging_level='Silent')).fit(X_train, y_train).predict(sub)

In [17]:
# Using two different stacked ensembles to make predictions using the vecstack stacking regressor
#
X = train.drop(['Square_ID'], axis = 1)
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 90)

sub = test.drop(['Square_ID'], axis = 1)

estimators_1 = [
    ('xgb', XGBRegressor(objective ='reg:squarederror')),
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor()),
    ('lgb', LGBMRegressor()),
    ('svr', SVR()),
    ('lasso', Lasso()),
    ('kneiba', KNeighborsRegressor()),
    ('cat', CatBoostRegressor(logging_level='Silent'))
]

stack = StackingTransformer(estimators_1, regression=True, verbose=0, metric =mean_squared_error, shuffle=True)
stack = stack.fit(X_train, y_train)
S_train = stack.transform(X_train)


final_estimator = CatBoostRegressor(logging_level='Silent')
final_estimator = final_estimator.fit(S_train, y_train)

S_sub = stack.transform(sub)
predictions_vecstack = final_estimator.predict(S_sub)



estimators_2 = [
    ('XBRF', XGBRFRegressor(objective ='reg:squarederror')),
    ('Bayesian', BayesianRidge()),
    ('ExtraTrees', ExtraTreesRegressor()),
    ('HistGradient', HistGradientBoostingRegressor()),
    ('NuSVR', NuSVR()),
    ('Ridge', Ridge()),
    ('KNeiba', KNeighborsRegressor()),
    ('cat', CatBoostRegressor(logging_level='Silent'))
]

stack = StackingTransformer(estimators_2, regression=True, verbose=0, metric =mean_squared_error, shuffle=True)
stack = stack.fit(X_train, y_train)
S_train = stack.transform(X_train)


final_estimator = CatBoostRegressor(logging_level='Silent')
final_estimator = final_estimator.fit(S_train, y_train)

S_sub = stack.transform(sub)
predictions_vecstack_2 = final_estimator.predict(S_sub)

In [19]:
# Averaging the two stacked predictions from sklearn and vecstack in the ratio of 9:1
#
predictions_vecstack = [x*0.9 + y*0.1 for x, y in zip(predictions_vecstack, predictions_vecstack_2)]
predictions_sreg = [x*0.9 + y*0.1 for x, y in zip(predictions_sreg, predictions_sreg_2)]


# Blending the two ensemble models and the catboost single model
#
stack = [x*0.3 + y*0.7 for x, y in zip(predictions_vecstack, predictions_sreg)]
stack_2 = [x*0.9 + y*0.1 for x, y in zip(stack, predictions_cat)]



# Retraining the models using the test data as training data and the predictions as the target
#
X = sub.copy()
y = stack_2

ridge = Ridge()
ridge.fit(X, y)
preds_ridge = ridge.predict(X)

cat = CatBoostRegressor(verbose = False)
cat.fit(X, y)
preds_cat = cat.predict(X)
# Blending the two trained models
#
blended_1 = [x*0.5 +y*0.5 for x, y in zip(preds_ridge, preds_cat)]



# Retrainig the models using the above approach but using different weights
#
stack = [x*0.4 + y*0.6 for x, y in zip(predictions_vecstack, predictions_sreg)]
stack_2 = [x*0.8 + y*0.2 for x, y in zip(stack, predictions_cat)]


X = sub.copy()
y = stack_2

ridge = Ridge()
ridge.fit(X, y)
preds_ridge = ridge.predict(X)

cat = CatBoostRegressor(verbose = False)
cat.fit(X, y)
preds_cat = cat.predict(X)

blended_2 = [x*0.5 +y*0.5 for x, y in zip(preds_ridge, preds_cat)]

blended_3 = [x*0.9 + y*0.1 for x, y in zip(blended_1, blended_2)]


# Further generalising the model by training using the simple Linear regression model
# Complementing it with the catboost model
#
X = sub.copy()
y = blended_3

linear = LinearRegression()
linear.fit(X, y)
preds_linear = linear.predict(X)

cat = CatBoostRegressor(verbose = False)
cat.fit(X, y)
preds_cat = cat.predict(X)


# Blending the two model predictions
# Creating a predictions file to be used in the next step, as you will have to restart the kernel
#
final_blend_1 = [x*0.1 + y*0.1 + z*0.8 for x, y, z in zip(preds_linear, preds_cat, blended_3)]
sub_df = pd.DataFrame({'Square_ID': test.Square_ID, 'target': final_blend_1}) 
sub_df.to_csv('final_blend_1.csv', index = False)